In [ ]:
%matplotlib inline

I have been trying to make SCAM give reasonable results with the forcings from NGAqua, but I have not had much success. I configure the SCAM model by calling 

```
$camcfg/configure -scam  -nlev 30 -fc gfortran \
   -nospmd -nospm -dyn eul \
   -res 1x1 -fflags '-fbacktrace -fcheck=all' \
   -chem none -ocn aquaplanet
```

I am providing the model data using the Intensive Observation Period (IOP) netCDF format described [here](http://www.cesm.ucar.edu/models/atm-cam/docs/scam/).

# CAM Included datasets

I first test the performance of the SCAM on some of the sample IOP datasets included with CAM.

In [ ]:
from xnoah import centderiv, integrate, swap_coord
from lib.cam import load_cam
from lib.thermo import mass_integrate

import glob, re


def decode_date(units):
    match = re.search(r"(\w+) since (\d\d+)-(\d\d)-(\d\d) (\d\d:\d\d:\d\d)",
                      units)
    
    unit = match.group(1)
    y = match.group(2)
    
    if len(y) == 2:
        y = '19' + y
    elif y[:2] == '00':
        y = '19' + y[2:]
    m = match.group(3)
    d = match.group(4)
    time =match.group(5)
    
    return f"{unit} since {y}-{m}-{d} {time}"
    

def load_cam(files):
    if isinstance(files, str):
        files = glob.glob(files)
    # load data

    ds = xr.auto_combine([xr.open_dataset(f, decode_times=False)
                          for f in files[:-1]], concat_dim='time')\
           .sortby('time')
    ds.time.attrs['units'] = decode_date(ds.time.units)
    ds = xr.decode_cf(ds)
    return ds

decode_date('days since 0069-06-25 00:00:00')


In this section, I plot the difference between humidity from SCAM, and the observed humidity.

## BOMEX

In [ ]:
bomex = load_cam("../ext/scam/run/bomex/camrun.cam.h0.*.nc")
bomex['QDIFF'].squeeze().T.plot.contourf()
plt.xticks(rotation=30)
plt.gca().invert_yaxis()

Why does the model dry out soo much!

## TWP

In [ ]:
twp = load_cam("../ext/scam/run/twp/camrun.cam.h0.*.nc")
twp['QDIFF'].squeeze().T.plot.contourf()
plt.xticks(rotation=45)
plt.gca().invert_yaxis()

In both the BOMEX and TWP cases, SCAM completely dries out the lower troposphere. After only 5 days in both cases there is nearly a -10 g/kg error in humidity below 800 hPa.

If we look at the time series of precipitable water (PW), we see a similar trend.

In [ ]:
PW = integrate(twp['Q'], 'lev')*100/9.81
PW.plot(label='CAM')

PW1 = integrate(twp.Q - twp.QDIFF, 'lev')*100/9.81
PW1.plot(label='observed')
plt.legend()

The surface temperature substantially decrease in this simulation. Why does this happen?

In [ ]:
twp['T'].isel(lev=-1).plot()
(twp['T']-twp.TDIFF).isel(lev=-1).plot()

This huge drop in temperature does not make sense. The sum of horizontal and vertical advection tedencies is positive there.

# Moisture budget

Why is this drying occuring? Let's look at the forcing file to find out.

In [ ]:
iop = xr.open_dataset("../ext/scam/run/twp/TWP06_4scam.nc")
iop = swap_coord(iop, {'time': 'tsec'})

In [ ]:
prec = iop.Prec*86400
evap = iop.lhflx*86400/2.51e6
integrated_adv = integrate(iop.divq+iop.vertdivq, 'lev')/9.81*86400

def time_mean(x):
    return integrate(x, 'tsec')/(x.tsec[-1] - x.tsec[0])

adv_time_mean = float(time_mean(integrated_adv))
prec_time_mean = float(time_mean(prec))
evap_time_mean = float(time_mean(evap))

Here are these quantities in mm/day.

In [ ]:
adv_time_mean, prec_time_mean, evap_time_mean

As we can see these terms approximately balance. The imbalance is only around .5 mm/day, which cannot possible explain the huge drop in PW seen in the single column model run.

In [ ]:
cam_prec =(twp.PRECC + twp.PRECL)*1000*86400
cam_prec.plot()

In [ ]:
(iop.Prec*86400).plot()

We can see CAMs precipitation matches the observed precip fairly well. After looking at CAM, I am pretty sure that it is not adding the vertical moisture advection terms.

In [ ]:
divq3d_int = integrate(twp.DIVQ3D, 'lev')*100/9.81 *86400
divq3d_int.mean('time')

It seems like the right variable is in there, but there is probably not any advection actually happening in the model.

In [ ]:
twp.DCQ.plot()

# NGAQUA

I realizse that I need to include the adiabatic term in divT.

In [ ]:
iop = xr.open_dataset("../data/processed/iop/0-8/iop.nc")
ngaqua = xr.open_dataset("../data/processed/iop/0-8/cam.nc")

In [ ]:
(ngaqua.PRECC*86400*1000).plot()

In [ ]:
(integrate(ngaqua.DIVQ3D, 'lev')/9.81 * 86400*100).plot()

In [ ]:
(integrate(iop.Q_dten, 'lev')/9.81 * 86400).mean('tsec')

In [ ]:
ngaqua['Q'].T.squeeze().plot.contourf(levels=np.arange(11)*.002)
plt.gca().invert_yaxis()
plt.xticks(rotation=30)

In [ ]:
iop['q'].T.squeeze().plot.contourf(levels=np.arange(11)*.002)
plt.gca().invert_yaxis()